<a href="https://colab.research.google.com/github/Adil1979/P6_Tag_Recommendation_Stackoverflow/blob/master/02_Supervised_Modelisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

## **1.Librairies**

In [2]:
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
#import chart_studio.plotly.plotly as py
from collections import Counter
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
#from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
random_state = 747

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **2.Data**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
X_questions = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/X_tfidf.csv')
y_tags = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/y_onehot.csv')

In [6]:
X_questions = X_questions.drop(['Unnamed: 0'], axis=1).iloc[:, 1:]
X_questions.head()

,unity,developer,able,generic,type,parameter,derived,object,looking,answer,question,come,tell,following,difference,constraint,explicitly,state,method,public,static,lt,gt,original,position,rotation,happen,angular,component,class,dom,element,believe,service,pas,root,node,e,g,foo,...,resolved,intermediate,gender,cipher,gl,polygon,acc,ch,airflow,dag,nio,rect,bot,myproject,openssl,venv,cxx,stroke,sf,holder,artifact,patch,fabric,dimen,cart,upper,locale,st,vertex,mpl,dbo,gm,celery,ca,hr,cran,defaultbuildoperationexecutor,grpc,xsd,gyp
0,0.0,0.0,0.0,0.0,0.160154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.042762,0.0,0.0,0.0,0.0,0.056064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
y_tags = y_tags.drop(['Unnamed: 0'], axis=1)
y_tags.columns = y_tags.columns.str.replace("Tag_st_", "")
y_tags.head()

,python pandas,javascript,python,c++,r,python pandas dataframe,javascript reactjs,python python-3.x,java,dart flutter,angular,html css,flutter,typescript,r ggplot2,c#,android,reactjs,git,php laravel,haskell,react-native,javascript jquery,javascript html,python numpy,angular typescript,javascript html css,flutter dart,sql sql-server,ios swift,rust,docker,kotlin,visual-studio-code,javascript node.js,julia,python matplotlib,c++ c++11,raku,r dplyr,...,c++ types,amazon-web-services aws-lambda aws-api-gateway,android kotlin dagger-2,java swing jframe jpanel jbutton,javascript push-notification,javascript firebase firebase-authentication,mysql mysql-workbench,javascript destructuring,r ggplot2 density-plot,c# powershell,python-2.7,ios xcode swiftui,python numpy random random-seed,ios app-store,python-3.x python-imaging-library,c# entity-framework-core ef-core-2.2,asp.net crystal-reports,c++ c++11 stl,python pdf reportlab,html angular,java tomcat,gtk3 vala,keycloak,node.js sass gulp gulp-sass,mysql laravel eloquent,python csv,c# asp.net-mvc entity-framework,visual-studio-code vscode-extensions vscode-tasks,c# sql sql-server linq,docker docker-compose dockerfile,c++ templates inheritance,c++ templates gcc,python rounding,r dplyr tidyeval,android kotlin android-jetpack android-jetpack-navigation,java gradle build,reactjs react-bootstrap formik,java ssl,javascript jquery datatable,angular primeng primeng-turbotable
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **3.Modelisation**

In [8]:
from sklearn.model_selection import train_test_split

from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

### **Train Test Split**

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_questions, y_tags,
                                                    test_size=0.3, 
                                                    random_state=random_state)

In [10]:
X_train.head()

,unity,developer,able,generic,type,parameter,derived,object,looking,answer,question,come,tell,following,difference,constraint,explicitly,state,method,public,static,lt,gt,original,position,rotation,happen,angular,component,class,dom,element,believe,service,pas,root,node,e,g,foo,...,resolved,intermediate,gender,cipher,gl,polygon,acc,ch,airflow,dag,nio,rect,bot,myproject,openssl,venv,cxx,stroke,sf,holder,artifact,patch,fabric,dimen,cart,upper,locale,st,vertex,mpl,dbo,gm,celery,ca,hr,cran,defaultbuildoperationexecutor,grpc,xsd,gyp
15114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.467604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.10072,0.0,0.0,0.066112,0.0,0.0,0.05486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.021629,0.0,0.00000,0.0,0.0,0.028357,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.036378,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.194445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y_train.head()

,python pandas,javascript,python,c++,r,python pandas dataframe,javascript reactjs,python python-3.x,java,dart flutter,angular,html css,flutter,typescript,r ggplot2,c#,android,reactjs,git,php laravel,haskell,react-native,javascript jquery,javascript html,python numpy,angular typescript,javascript html css,flutter dart,sql sql-server,ios swift,rust,docker,kotlin,visual-studio-code,javascript node.js,julia,python matplotlib,c++ c++11,raku,r dplyr,...,c++ types,amazon-web-services aws-lambda aws-api-gateway,android kotlin dagger-2,java swing jframe jpanel jbutton,javascript push-notification,javascript firebase firebase-authentication,mysql mysql-workbench,javascript destructuring,r ggplot2 density-plot,c# powershell,python-2.7,ios xcode swiftui,python numpy random random-seed,ios app-store,python-3.x python-imaging-library,c# entity-framework-core ef-core-2.2,asp.net crystal-reports,c++ c++11 stl,python pdf reportlab,html angular,java tomcat,gtk3 vala,keycloak,node.js sass gulp gulp-sass,mysql laravel eloquent,python csv,c# asp.net-mvc entity-framework,visual-studio-code vscode-extensions vscode-tasks,c# sql sql-server linq,docker docker-compose dockerfile,c++ templates inheritance,c++ templates gcc,python rounding,r dplyr tidyeval,android kotlin android-jetpack android-jetpack-navigation,java gradle build,reactjs react-bootstrap formik,java ssl,javascript jquery datatable,angular primeng primeng-turbotable
15114,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27789,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9638,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20646,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### **OneVsRestClass and GridSearchCV**

In [12]:
def GridSearch(model, hyperparameters, X_tr, y_tr):
  classifier = OneVsRestClassifier(model)
  gs = GridSearchCV(classifier, hyperparameters, cv=5, n_jobs = -1, scoring= 'accuracy')
  gs.fit(X_tr, y_tr)
  return gs

### **Logistic Regression**

In [ ]:
lr_model = LogisticRegression()
lr_hyperparameters = dict(estimator__C=[10**-2, 10**0, 10**2])
lr_cv = GridSearch(lr_model, lr_hyperparameters, X_train, y_train)

In [1]:
lr_cv.best_params_

NameError: ignored

In [ ]:
lr_cv.score(X_test, y_test)

### **Random Forest**

In [ ]:
RF_model = RandomForestClassifier()
rf_hyperparameters = dict(rf__n_estimators=[10, 30, 50], rf_criterion=['gini', 'entropy'])
rf_cv = GridSearch(rf_model, rf_hyperparameters, X_train, y_train)

In [ ]:
rf_cv.best_params_

In [ ]:
rf_cv.score(X_test, y_test)

### **Native Bayes**

In [ ]:
nb_model = MultinomialNB()
nb_hyperparameters = dict(nb__estimator__alpha=[10**-2, 10**0, 10**2])
nb_cv = GridSearch(nb_model, nb_hyperparameters, X_train, y_train)

In [ ]:
nb_cv.best_params_

In [ ]:
np_cv.score(X_test, y_test)